<a href="https://colab.research.google.com/github/nywkim/project/blob/main/AI_06_%EA%B9%80%EC%98%81%EC%9A%B0_P4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

### Project 4
## 간단한 음원 데이터 장르 분류 모델 만들기
#### 대략적인 데이터 셋의 정보와 음원 데이터를 분석하는 데 알아야 할 지식들을 살펴보고, 전처리 이후 모델을 만들어 비교해 보자.

In [ ]:
import librosa, IPython
import librosa.display as lplt
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from sklearn import preprocessing
seed = 99
np.random.seed(seed)

In [ ]:
# 오디오를 wav 파일로 변환 후 저장
!pip install pydub
from pydub import AudioSegment

dst = "test.wav"

audSeg = AudioSegment.from_file('drive/MyDrive/Love&Evil.m4a')
# mp3파일은 audSeg = AudioSegment.from_mp3(src)
audSeg.export(dst, format="wav")

In [ ]:
# 오디오 부착
import os
wav = 'test.wav'
(file_dir, file_id) = os.path.split(wav)
print("file_id:", file_id)

In [ ]:
import wave
import array
w = wave.open(os.path.join(os.getcwd(), "test.wav"), "r")

print('음원의 프레임 수 :', w.getnframes())
print('음원의 초당 프레임 수 :', w.getframerate())
print('음원의 채널 수 :', w.getnchannels())

wavLen = w.getnframes() / w.getframerate()
# 길이 = 음원의 프레임 수 / 음원의 초당 프레임 수

buffer = w.readframes(w.getnframes())
# 해당 음원을 음원의 프레임 수만큼 프레임을 읽어 버퍼로 로드

amplitude = (np.frombuffer(buffer, dtype="int16"))
# 로드한 버퍼를 int16형식의 데이터타입을 담은 리스트를 amplitude 변수에 담는다.
# 진폭배열의 길이 = 프레임 수 * 채널 수

In [ ]:
# 음원의 시작과 끝 시간 설정
startsec = 3
endsec = 33
s_amp = amplitude[int(startsec * w.getframerate() * w.getnchannels()):int(endsec * w.getframerate() * w.getnchannels())]
# "시작 초 (=startsec) * 프레임 수 * 채널 수 ~ 끝 초 (=endsec) * 프레임 수 * 채널 수"까지만 s_amp에 담기.

In [ ]:
save_wave = wave.Wave_write(os.path.join("cut.wav"))
                    # 저장할 위치를 지정하는 함수.
save_wave.setparams(w.getparams())
                    # 파라미터를 설정하며, writeframes를 통해 프레임별로 들어갈 파형값을 배열에서 가져옴
save_wave.writeframes(array.array('h', s_amp).tobytes())
                    # array.array는 s_amp 리스트 안의 값을 제시되는 타입값으로 배열을 생성함.
save_wave.close()   # 스트림 close


##librosa를 통한 오디오 분석

In [ ]:
audio = 'cut.wav'
data, sr = librosa.load(audio) # 진폭을 시간 순서대로 나열한 것, Hz(1초당 샘플 단위)
audio_data, _ = librosa.effects.trim(data) # 오디오 시각화를 위한 손질

In [ ]:
IPython.display.Audio(audio_data, rate=sr)

In [ ]:
plt.figure(figsize=(15,5)) # 시각화 크기 설정
lplt.waveplot(audio_data)
plt.show()

In [ ]:
plt.figure(figsize=(15,5))
librosa.display.waveplot(data, sr, alpha=0.5)
plt.xlabel("Time (s)")
plt.ylabel("Amplitude")
plt.title("Waveform")

In [ ]:
# 푸리에 변환 : Time > Frequency, 진폭 > 크기 (분석을 위하여) 각 주파수가 얼마만큼의 크기를 차지하는지 나타냄.
# fft sample
fft = np.fft.fft(data)

magnitude = np.abs(fft) 

f = np.linspace(0,sr,len(magnitude))

left_spectrum = magnitude[:int(len(magnitude) / 2)]
left_f = f[:int(len(magnitude) / 2)]

plt.figure(figsize=(10,5))
plt.plot(left_f, left_spectrum)
plt.xlabel("Frequency")
plt.ylabel("Magnitude")
plt.title("Power spectrum")

In [ ]:
# STFT는 음성을 일정 작은 프레임으로 잘라 시간순으로 붙여 푸리에 변환을 한 것.
hop_length = 512
n_fft = 2048

Fourier = np.abs(librosa.stft(data, n_fft=n_fft, hop_length=hop_length))
plt.figure(figsize=(10,5))
plt.plot(Fourier)
plt.show()

In [ ]:
# Spectrogram : time과 frequency 사이 음의 세기를 dB로 변환하여 색으로 표현
hop_length_duration = float(hop_length) / sr
n_fft_duration = float(n_fft) / sr

stft = librosa.stft(data, n_fft=n_fft, hop_length=hop_length)
magnitude = np.abs(stft)
log_spectrogram = librosa.amplitude_to_db(magnitude, ref=np.max)

plt.figure(figsize=(10,5))
librosa.display.specshow(log_spectrogram, sr=sr, x_axis='time', hop_length=hop_length)
plt.xlabel("Time")
plt.ylabel("Frequency")
plt.colorbar(format="%+2.0f dB")
plt.title("Spectrogram (dB)")

In [ ]:
"""
Mel_spectrogram 만들기
우리는 인접한 주파수와 높은 주파수 대역은 잘 구분하지 못하기 때문에,
이런 인간의 청각적 지각 능력에 맞춰서 로그 스케일로 스펙트로그램의 주파수 축을 줄이고
이 값들을 몇 개의 주파수 대역대로 묶으면 크기를 줄이는 동시에 가장 중요한 정보들을 보존할 수 있는 멜 스펙트로그램(mel-spectrogram)을 만들 수 있다,
"""
mel_spec = librosa.feature.melspectrogram(data, sr=sr)
mel_spec_db = librosa.amplitude_to_db(mel_spec, ref=np.max)

plt.figure(figsize=(16,6))
lplt.specshow(mel_spec_db, sr=sr, hop_length=hop_length, x_axis='time', y_axis='log')
plt.colorbar()
plt.title("Mel Spectrogram")
plt.show()

In [ ]:
# BPM 추출
tempo, _ = librosa.beat.beat_track(data, sr=sr)
print('BPM :', tempo)

# Zero Crossing Rate (mean, var) 음파가 0 값을 지나치는 비율
zero_c = librosa.zero_crossings(data, pad=False)
zero_crossing_rate_mean = np.mean(zero_c)
print('zero_crossing_rate_mean :', zero_crossing_rate_mean)
zero_crossing_rate_var = np.var(zero_c)
print('zero_crossing_rate_var :', zero_crossing_rate_var)

In [ ]:
# Harmonics(사람이 귀로 구분하기 힘든 음악의 특징들)
# Percussives(리듬, 감정을 나타내는 충격파)
harm, perc = librosa.effects.hpss(data)

harmony_mean = np.mean(harm)
harmony_var = np.var(harm)
perceptr_mean = np.mean(perc)
perceptr_var = np.var(perc)

print('harmony (mean, var) & percussive (mean, var)')
print(harmony_mean, harmony_var), print(perceptr_mean, perceptr_var)

plt.figure(figsize=(16,6))
plt.plot(harm, color='g')
plt.plot(perc, color='r')
plt.show()

In [ ]:
# Spectral Centroid : 소리의 중심이 어디에 크게 실려있는지 확인.
spec_centroids = librosa.feature.spectral_centroid(data, sr=sr)[0]
spectral_centroid_mean = np.mean(spec_centroids)
spectral_centroid_var = np.var(spec_centroids)

frames = range(len(spec_centroids))
t = librosa.frames_to_time(frames)

plt.figure(figsize=(16,6))
librosa.display.waveplot(data, sr=sr, alpha=0.5, color='b')
plt.plot(t, preprocessing.minmax_scale(spec_centroids), color='r') # 0-1사이로 맞춰 디스플레이
plt.show()

In [ ]:
# Spectral Rolloff : 낮은 주파수에 어느 부분이 얼마나 집중되었는지.
spec_rolloff = librosa.feature.spectral_rolloff(data, sr=sr)[0]
rolloff_mean = np.mean(spec_rolloff)
rolloff_var = np.var(spec_rolloff)

plt.figure(figsize=(16,6))
librosa.display.waveplot(data, sr=sr, alpha=0.5, color='b')
plt.plot(t, preprocessing.minmax_scale(spec_rolloff), color='r') # 0-1사이로 맞춰 디스플레이
plt.show()

In [ ]:
# 음계와 관련한 부분을 시각화
chroma_stft = librosa.feature.chroma_stft(data, sr=sr)
chroma_stft_mean = np.mean(chroma_stft)
chroma_stft_var = np.var(chroma_stft)

plt.figure(figsize=(16,6))
lplt.specshow(chroma_stft, sr=sr, x_axis='time', y_axis='chroma', cmap='coolwarm')
plt.colorbar()
plt.title("Chroma Features")
plt.show()

In [ ]:
# 오디오 샘플에서 각 프레임에 대한 RMS를 계산하는 함수
rms = librosa.feature.rms(data)
rms_mean = np.mean(rms)
rms_var = np.var(rms)

spec_bw = librosa.feature.spectral_bandwidth(data, sr=sr)
spectral_bandwidth_mean = np.mean(spec_bw)
spectral_bandwidth_var = np.mean(spec_bw)

In [ ]:
# 음악 장르 분석, 음성 인식 등 다양한 분야에서 사용되는 특성 추출법 MFCC
MFCCs = librosa.feature.mfcc(data, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mfcc=39)

# display MFCCs
plt.figure(figsize=(16,6))
librosa.display.specshow(MFCCs, sr=sr, hop_length=hop_length)
plt.xlabel("Time")
plt.ylabel("MFCC coefficients")
plt.colorbar()
plt.title("MFCCs")

# show plots
plt.show()

# 데이터 셋을 준비합니다.

In [ ]:
m3 = pd.read_csv('drive/MyDrive/kaggle/Data/features_3_sec.csv')

In [ ]:
m3.head()

In [ ]:
print("데이터(3초) 형태 :",m3.shape)
print("총 10 클래스의 장르들과 각 장르별 데이터 수 :")
m3['label'].value_counts()

In [ ]:
# 각 mean column들의 히트맵 
spike_cols = [col for col in m3.columns if 'mean' in col]
corr = m3[spike_cols].corr()

# 대각선 기준 윗면 삼각형 가리기
mask = np.triu(np.ones_like(corr, dtype=np.bool))

# 사이즈 설정 + 팔레트
f, ax = plt.subplots(figsize=(16, 11));
cmap = sns.diverging_palette(150, 275, s=80, l=55, n=9)

# 히트맵 그리기
sns.heatmap(corr, cmap=cmap, vmax=.3, center=0, mask=mask,
            square=True, linewidths=.5, cbar_kws={"shrink": .5})

plt.title('Correlation Heatmap (only "mean" variables)', fontsize = 20)
plt.xticks(fontsize = 10)
plt.yticks(fontsize = 10);

In [ ]:
label_index = dict()
index_label = dict()
for i, x in enumerate(m3.label.unique()):
    label_index[x] = i
    index_label[i] = x
print(index_label)
m3.label = [label_index[l] for l in m3.label]

In [ ]:
features = m3.drop(['label','filename','length'],axis=1)
target = m3['label'] 

In [ ]:
cols = features.columns
min_max_scaler = preprocessing.MinMaxScaler()
np_scaled = min_max_scaler.fit_transform(features)

features = pd.DataFrame(np_scaled, columns = cols)

In [ ]:
features # 정규화 완료

In [ ]:
from sklearn.model_selection import train_test_split

X_train, test1, y_train, test2 = train_test_split(features, target, test_size=0.3, random_state=seed)
print(X_train.shape, y_train.shape)
X_val, X_test, y_val, y_test = train_test_split(test1, test2, test_size=0.2, random_state=seed)
X_val.shape, X_test.shape, y_val.shape, y_test.shape

In [ ]:
import tensorflow as tf
import tensorflow.keras as keras
from keras.callbacks import EarlyStopping,LearningRateScheduler
from keras import Sequential
from keras.layers import *
tf.random.set_seed(seed)

In [ ]:
# Deep Learning Model 만들기
model = Sequential()

model.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model.add(Dropout(0.2))
model.add(Dense(256, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(64, activation='relu'))
model.add(Dropout(0.2))
model.add(Dense(32, activation='relu'))
model.add(Dense(10, activation='softmax'))
model.summary()

In [ ]:
early_stopping= EarlyStopping(monitor='val_loss',mode='min',verbose=1,patience=75)

In [ ]:
batch_size = 128

model.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics='accuracy')
a = model.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=150, 
                     batch_size=batch_size, callbacks=[early_stopping])

In [ ]:
print("Max. Validation Accuracy",max(a.history["val_accuracy"]))
pd.DataFrame(a.history).plot(figsize=(12,6))
plt.show()

In [ ]:
model2 = Sequential()

model2.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model2.add(Dropout(0.2))
model2.add(Dense(256, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(64, activation='relu'))
model2.add(Dropout(0.2))
model2.add(Dense(10, activation='softmax'))
model2.summary()

In [ ]:

model2.compile(optimizer='adam',
                  loss='sparse_categorical_crossentropy',
                  metrics='accuracy')
a2 = model2.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=150, 
                     batch_size=batch_size, callbacks=[early_stopping])

In [ ]:
print("Max. Validation Accuracy",max(a2.history["val_accuracy"]))
pd.DataFrame(a2.history).plot(figsize=(12,6))
plt.show()

In [ ]:
model3 = Sequential()

model3.add(Dense(512, activation='relu', input_shape=(X_train.shape[1],)))
model3.add(Dropout(0.2))
model3.add(Dense(256, activation='relu'))
model3.add(Dropout(0.2))
model3.add(Dense(64, activation='relu'))
model3.add(Dropout(0.2))
model3.add(Dense(10, activation='softmax'))
model3.summary()

In [ ]:

model3.compile(optimizer='sgd',
                  loss='sparse_categorical_crossentropy',
                  metrics='accuracy')
a3 = model3.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=500, 
                     batch_size=batch_size, callbacks=[early_stopping])

In [ ]:
model4 = Sequential()

model4.add(Dense(1024, activation='relu', input_shape=(X_train.shape[1],)))
model4.add(Dropout(0.2))
model4.add(Dense(512, activation='relu'))
model4.add(Dropout(0.2))
model4.add(Dense(256, activation='relu'))
model4.add(Dropout(0.2))
model4.add(Dense(64, activation='relu'))
model4.add(Dropout(0.2))
model4.add(Dense(10, activation='softmax'))
model4.summary()

In [ ]:
model4.compile(optimizer='rmsprop',
                  loss='sparse_categorical_crossentropy',
                  metrics='accuracy')
a4 = model4.fit(X_train, y_train, validation_data=(X_val, y_val), epochs=500, 
                     batch_size=batch_size, callbacks=[early_stopping])

In [ ]:
print("Max. Validation Accuracy",max(a4.history["val_accuracy"]))
pd.DataFrame(a4.history).plot(figsize=(12,6))
plt.show()

In [ ]:
test_loss, test_acc  = model4.evaluate(X_test, y_test, batch_size=128)
print("The test Loss is :",test_loss)
print("\nThe Best test Accuracy is :",test_acc*100)

In [ ]:
def predict(model, X, y):
    #X = X[np.newaxis,...]
    prediction = model.predict(X)
    predicted_index = np.argmax(prediction, axis=1)
    print(f"Expected label:")
    print(f"{y}, Predicted label: {predicted_index}")
predict(model4, X_test.head(), y_test.head())

spectrogram 이미지로 딥러닝 적용해보기 + 더 좋은 데이터셋 등 보완해야 할 점이 있습니다.

In [ ]:
from xgboost import XGBClassifier
from sklearn.metrics import accuracy_score
xgb = XGBClassifier(n_estimators=1000, learning_rate=0.05)
xgb.fit(X_train, y_train)
preds = xgb.predict(X_val)
print('Accuracy', 'XGB Classifier', ':', round(accuracy_score(y_val, preds), 5), '\n')